In [3]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
# Set Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU for better performance
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource issues

In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

class Score:
    team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
    team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bengaluru", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

    def __init__(self,url):
        self.url = url
        self.player_list,self.winner,self.man_of_the_match,self.innings_list,self.batsmen_list,self.match_score,self.did_not_bat,self.bowlers_info = self.scorecard()

    def find_team(self,text):
        def check_list(team_list):
            matches = [word for word in team_list if word in text]
            if len(matches) > 1:
                raise ValueError(f"Error: More than one match found: {matches}")
            elif len(matches) == 1:
                return matches[0]
            return None

        match = check_list(self.team_names_sf)
        if match:
            return self.team_names_ff[self.team_names_sf.index(match)]
        match = check_list(self.team_names_ff)
        if match:
            return match
        raise ValueError("Error: No match found in either list")

    def scorecard(self):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }

        response = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Initialize lists to store data for the match
        innings_list = []
        batsmen_list = pd.DataFrame()
        match_score = pd.DataFrame()
        did_not_bat = pd.DataFrame()
        bowlers_info = pd.DataFrame()
        player_list = pd.DataFrame()

        winner = soup.find('p',class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo").text.strip()
        winner = self.find_team(winner)

        match_details_table = soup.find('table',class_= "ds-w-full ds-table ds-table-sm ds-table-auto") 
        table_rows = match_details_table.find_all('tr')
        for row in table_rows:
            if "Player Of The Match" in row.text:
                man_of_the_match = row.find('td',class_="ds-text-typo").text.strip()
        
        # Find all divs with class "ds-rounded-lg ds-mt-2" (which contain innings tables)
        innings_tables = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')

        # Loop through each innings table div
        for innings_table in innings_tables:

            innings_number = innings_tables.index(innings_table) + 1

            # Find the div that contains the team name and innings number
            team_innings_div = innings_table.find('div', class_='ds-flex ds-px-4 ds-border-b ds-border-line ds-py-3 ds-bg-ui-fill-translucent-hover')
            batting_innings = team_innings_div.text.strip().replace('\xa0',' ')
            batting_innings = self.find_team(batting_innings)
                
            innings_list.append(batting_innings)

            
            batsmen_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
        
            # Batsmen details extraction
            batting_info = batsmen_table.find('tbody')
            batting_info = batting_info.find_all('tr')

            for batter in batting_info:
                try:
                    batsman_stats = batter.find_all('td')
                    name = batsman_stats[0].text.strip()
                    if '†' in name:
                        name = name.replace('†','').strip()
                    if '(c)' in name:
                        name = name.replace('(c)','').strip()
                    dismissal = batsman_stats[1].text.strip()
                    runs = int(batsman_stats[2].text.strip())
                    balls = int(batsman_stats[3].text.strip())
                    fours = int(batsman_stats[5].text.strip())
                    sixes = int(batsman_stats[6].text.strip())
                    strike_rate = float(batsman_stats[7].text.strip())
                            
                    # Append the details to the list
                    batsmen_stat = {'Innings Number': innings_number,
                        'Innings Name': batting_innings,
                        'Batsman': name,
                        'Dismissal': dismissal,
                        'Runs': runs,
                        'Balls': balls,
                        '4s': fours,
                        '6s': sixes,
                        'Strike Rate': strike_rate
                    }
                    batsmen_list = batsmen_list._append(batsmen_stat, ignore_index = True)
                    player_list = player_list._append({'Team':batting_innings, 'Player': name}, ignore_index = True)
                    d += 1
                except:
                    pass

            # Extract "Did Not Bat" information
            did_not_bat_row = batsmen_table.find('tr', class_='!ds-border-b-0')
            if did_not_bat_row:
                batsmen = did_not_bat_row.find_all('div', class_='ds-popper-wrapper ds-inline')
                for batsman in batsmen:
                    batsman_name = (batsman.get_text(strip=True)).replace(",", "")
                    if '†' in batsman_name:
                        batsman_name = batsman_name.replace('†','').strip()
                    if '(c)' in batsman_name:
                        batsman_name = batsman_name.replace('(c)','').strip()
                    did_not_bat = did_not_bat._append({'Innings Number': innings_number, 'Innings Name': batting_innings, 'Batsman':batsman_name}, ignore_index = True)
                    player_list = player_list._append({'Team':batting_innings, 'Player': batsman_name}, ignore_index = True)
                    
            #Extracting Bowling Info
            bowling_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')

            # Check if the bowling table is present
            if bowling_table:
                headers = []
                head_data = bowling_table.find_all('th')
                for bowler_header in head_data:
                    headers.append(bowler_header.text.strip())
                headers = list(map(lambda x: "Bowler" if x == "Bowling" else x, headers))
                headers = list(map(lambda x: "Overs" if x == "O" else x, headers))
                headers = list(map(lambda x: "Maidens" if x == "M" else x, headers))
                headers = list(map(lambda x: "Runs" if x == "R" else x, headers))
                headers = list(map(lambda x: "Wickets" if x == "W" else x, headers))
                headers = list(map(lambda x: "Economy" if x == "ECON" else x, headers))
                
                bowler_rows = bowling_table.find('tbody').find_all('tr')
                #try:
                for row in bowler_rows:
                    bowler_data = {}
                    bowler_data['Innings Number'] = innings_number
                    bowler_data['Innings Name'] = batting_innings
                    bowler_row_data = row.find_all('td')

                    try:
                        for i in range(len(headers)):
                            if headers[i] in ["Maidens","Runs", "Wickets","0s"]:
                                bowler_data[headers[i]] = int(bowler_row_data[i].text.strip())
                            elif headers[i] in ["Overs","Economy"]:
                                bowler_data[headers[i]] = float(bowler_row_data[i].text.strip())
                            elif headers[i] == "Bowler":
                                bowler_data[headers[i]] = (bowler_row_data[i].text.strip())

                        # Append the bowler's data to the list
                        bowlers_info = bowlers_info._append(bowler_data, ignore_index = True)
                    except:
                        pass
        return player_list, winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info
    
    def printing_scorecard(self):
        print("Player List:")
        print(self.player_list)
        print()
        for innings in self.innings_list:
            print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
            print(innings + ":")
            print()
        
            print("Batsmen:")
            print(self.batsmen_list[self.batsmen_list['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']))
            print()

            # Print "Did Not Bat" information)
            if self.did_not_bat.empty == False:
                if self.did_not_bat[self.did_not_bat['Innings Name'] == innings].empty == False:
                    print("Did Not Bat:")
                    print(self.did_not_bat[self.did_not_bat['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']))
                    print()

            print("Bowlers:")
            print(self.bowlers_info[self.bowlers_info['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']))
            print()
        
        print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ") 
        print("Winner: ",self.winner)
        print()
        print("Man of the Match: ",self.man_of_the_match)

url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-final-1426312/full-scorecard"
match1 = Score(url)
match1.printing_scorecard()
        

Player List:
                     Team              Player
0     Sunrisers Hyderabad     Abhishek Sharma
1     Sunrisers Hyderabad         Travis Head
2     Sunrisers Hyderabad      Rahul Tripathi
3     Sunrisers Hyderabad       Aiden Markram
4     Sunrisers Hyderabad  Nitish Kumar Reddy
5     Sunrisers Hyderabad    Heinrich Klaasen
6     Sunrisers Hyderabad       Shahbaz Ahmed
7     Sunrisers Hyderabad         Abdul Samad
8     Sunrisers Hyderabad         Pat Cummins
9     Sunrisers Hyderabad      Jaydev Unadkat
10    Sunrisers Hyderabad   Bhuvneshwar Kumar
11    Sunrisers Hyderabad         T Natarajan
12  Kolkata Knight Riders  Rahmanullah Gurbaz
13  Kolkata Knight Riders        Sunil Narine
14  Kolkata Knight Riders      Venkatesh Iyer
15  Kolkata Knight Riders        Shreyas Iyer
16  Kolkata Knight Riders         Rinku Singh
17  Kolkata Knight Riders       Andre Russell
18  Kolkata Knight Riders     Ramandeep Singh
19  Kolkata Knight Riders      Mitchell Starc
20  Kolkata Knight Ri

In [2]:
def find_team(text):
# Function to check matches in a given list
    def check_list(team_list):
        matches = [word for word in team_list if word in text]
        if len(matches) > 1:
            raise ValueError(f"Error: More than one match found: {matches}")
        elif len(matches) == 1:
            return matches[0]
        return None

    team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
    team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bangalore", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

    # Check in first list
    match = check_list(team_names_sf)
    if match:
        return team_names_ff[team_names_sf.index(match)]
    
    # Check in second list if no match found in first list
    match = check_list(team_names_ff)
    if match:
        return match

    raise ValueError("Error: No match found in either list")

In [3]:
def scorecard(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Initialize lists to store data for the match
    innings_list = []
    batsmen_list = pd.DataFrame()
    match_score = pd.DataFrame()
    did_not_bat = pd.DataFrame()
    bowlers_info = pd.DataFrame()

    winner = soup.find('p',class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo").text.strip()
    winner = find_team(winner)

    match_details_table = soup.find('table',class_= "ds-w-full ds-table ds-table-sm ds-table-auto") 
    table_rows = match_details_table.find_all('tr')
    for row in table_rows:
        if "Player Of The Match" in row.text:
            man_of_the_match = row.find('td',class_="ds-text-typo").text.strip()
    
    # Find all divs with class "ds-rounded-lg ds-mt-2" (which contain innings tables)
    innings_tables = soup.find_all('div', class_='ds-rounded-lg ds-mt-2')

    # Loop through each innings table div
    for innings_table in innings_tables:

        innings_number = innings_tables.index(innings_table) + 1

        # Find the div that contains the team name and innings number
        team_innings_div = innings_table.find('div', class_='ds-flex ds-px-4 ds-border-b ds-border-line ds-py-3 ds-bg-ui-fill-translucent-hover')
        batting_innings = team_innings_div.text.strip().replace('\xa0',' ')
        batting_innings = find_team(batting_innings)
            
        innings_list.append(batting_innings)

        
        batsmen_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
    
        # Batsmen details extraction
        batting_info = batsmen_table.find_all('tr', class_='')
        batsmen_stats = batting_info[:len(batting_info)-2]

        for batter in batsmen_stats:
            batsman_stats = batter.find_all('td')
            name = batsman_stats[0].text.strip()
            dismissal = batsman_stats[1].text.strip()
            runs = batsman_stats[2].text.strip()
            balls = batsman_stats[3].text.strip()
            minutes = batsman_stats[4].text.strip()
            fours = batsman_stats[5].text.strip()
            sixes = batsman_stats[6].text.strip()
            strike_rate = batsman_stats[7].text.strip()
                    
            # Append the details to the list
            batsmen_stat = {'Innings Number': innings_number,
                'Innings Name': batting_innings,
                'Batsman': name,
                'Dismissal': dismissal,
                'Runs': runs,
                'Balls': balls,
                'Minutes': minutes,
                '4s': fours,
                '6s': sixes,
                'Strike Rate': strike_rate
            }
            batsmen_list = batsmen_list._append(batsmen_stat, ignore_index = True)
        
        # Extract the total score, overs, and run rate
        total_row = batting_info[-2]  # The last 'tr' containing score info
        total_datapoints = total_row.find_all('td')
        if total_datapoints[0].text.strip() == "Total":
            score = total_datapoints[2].text.strip()
            other_info = total_datapoints[1].text.strip()
            overs = other_info.split(' Ov')[0]
            remaining = other_info.split('RR: ')[1].split(')')[0]
            if "Mins" not in remaining:
                run_rate = remaining
                mins = "-"
            else:
                run_rate = remaining.split(', ')[0]
                mins = remaining.split(', ')[1].split(" Mins")[0]                   
            # Add score, overs, and run rate to the match score list
            innings_score = {'Innings Number': innings_number,
                'Innings Name': batting_innings,
                'Score': score,
                'Overs': overs,
                'Run Rate': run_rate,
                "Minutes" : mins
            }
            match_score = match_score._append(innings_score, ignore_index = True)

        # Extract "Did Not Bat" information
        did_not_bat_row = batsmen_table.find('tr', class_='!ds-border-b-0')
        if did_not_bat_row:
            batsmen = did_not_bat_row.find_all('div', class_='ds-popper-wrapper ds-inline')
            for batsman in batsmen:
                batsman_name = (batsman.get_text(strip=True)).replace(",", "")
                did_not_bat = did_not_bat._append({'Innings Number': innings_number, 'Innings Name': batting_innings, 'Batsman':batsman_name}, ignore_index = True)

                
        #Extracting Bowling Info
        bowling_table = innings_table.find('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')

        # Check if the bowling table is present
        if bowling_table:
            headers = []
            head_data = bowling_table.find_all('th')
            for bowler_header in head_data:
                headers.append(bowler_header.text.strip())
            headers = list(map(lambda x: "Bowler" if x == "Bowling" else x, headers))
            headers = list(map(lambda x: "Overs" if x == "O" else x, headers))
            headers = list(map(lambda x: "Maidens" if x == "M" else x, headers))
            headers = list(map(lambda x: "Runs" if x == "R" else x, headers))
            headers = list(map(lambda x: "Wickets" if x == "W" else x, headers))
            headers = list(map(lambda x: "Economy" if x == "ECON" else x, headers))
            headers = list(map(lambda x: "Wides" if x == "WD" else x, headers))
            headers = list(map(lambda x: "No Balls" if x == "NB" else x, headers))
            
            bowler_rows = bowling_table.find('tbody').find_all('tr',class_='')

            for row in bowler_rows:
                bowler_data = {}
                bowler_data['Innings Number'] = innings_number
                bowler_data['Innings Name'] = batting_innings
                bowler_row_data = row.find_all('td')

                for i in range(len(headers)):
                    bowler_data[headers[i]] = bowler_row_data[i].text.strip()        

                # Append the bowler's data to the list
                bowlers_info = bowlers_info._append(bowler_data, ignore_index = True)
    
    #driver.quit()
    return winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info

In [5]:
def printing_scorecard(winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info):
    for innings in innings_list:
        print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
        print(innings + ":")
        print()
    
        print("Batsmen:")
        display(batsmen_list[batsmen_list['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))

        # Print Score Information
        print("Score:")
        display(match_score[match_score['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))
    
        # Print "Did Not Bat" information
        if did_not_bat.empty == False:
            if did_not_bat[did_not_bat['Innings Name'] == innings].empty == False:
                print("Did Not Bat:")
                display(did_not_bat[did_not_bat['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
                {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
            ]))
        
        print("Bowlers:")
        display(bowlers_info[bowlers_info['Innings Name'] == innings].drop(columns=['Innings Number', 'Innings Name']).style.hide(axis="index").set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # Center-align headers
            {'selector': 'td', 'props': [('text-align', 'center')]}   # Center-align data
        ]))
    print("Winner: ",winner)
    print()
    print("Man of the Match: ",man_of_the_match)


In [7]:
#match_url = input("Enter the match url: ")
match_url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-final-1426312/full-scorecard"

begin = time.time() 
winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info = scorecard(match_url)

printing_scorecard(winner, man_of_the_match, innings_list, batsmen_list, match_score, did_not_bat, bowlers_info)
end = time.time()
# total time taken 
total_time_taken = end-begin
minutes = str(int(total_time_taken/60))
seconds = str(round(total_time_taken % 60,3))
total_time_taken = minutes+"m "+seconds+"s"
print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")  
print(f"Total runtime of the program is {total_time_taken}")             

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Sunrisers Hyderabad:

Batsmen:


Batsman,Dismissal,Runs,Balls,Minutes,4s,6s,Strike Rate
Abhishek Sharma,b Starc,2,5,3,0,0,40.00
Travis Head,c †Rahmanullah Gurbaz b Arora,0,1,11,0,0,0.00
Rahul Tripathi,c Ramandeep Singh b Starc,9,13,21,1,0,69.23
Aiden Markram,c Starc b Russell,20,23,41,3,0,86.95
Nitish Kumar Reddy,c †Rahmanullah Gurbaz b Harshit Rana,13,10,14,1,1,130.00
Heinrich Klaasen †,b Harshit Rana,16,17,32,1,0,94.11
Shahbaz Ahmed,c Narine b Varun,8,7,5,0,1,114.28
Abdul Samad,c †Rahmanullah Gurbaz b Russell,4,4,3,0,0,100.00
Pat Cummins (c),c Starc b Russell,24,19,28,2,1,126.31
Jaydev Unadkat,lbw b Narine,4,11,16,0,0,36.36


Score:


Score,Overs,Run Rate,Minutes
113,18.3,6.10,-


Did Not Bat:


...

Did Not Bat:


Batsman
Rinku Singh
Andre Russell
Ramandeep Singh
Mitchell Starc
Vaibhav Arora
Harshit Rana
Varun Chakravarthy


Bowlers:


Bowler,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,Wides,No Balls
Bhuvneshwar Kumar,2,0,25,0,12.50,5,2,2,1,0
Pat Cummins,2,0,18,1,9.00,7,2,1,1,0
T Natarajan,2,0,29,0,14.50,2,4,1,2,0
Shahbaz Ahmed,2.3,0,22,1,8.80,7,1,2,0,0
Jaydev Unadkat,1,0,9,0,9.00,1,1,0,0,0
Aiden Markram,1,0,5,0,5.00,2,0,0,0,0


Winner:  Kolkata Knight Riders

Man of the Match:  Mitchell Starc
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Total runtime of the program is 0m 0.424s


In [18]:
print(innings_list)

['Sunrisers Hyderabad', 'Kolkata Knight Riders']


In [67]:
display(batsmen_list)
# batsmen_list.loc[batsmen_list['Innings Number'] == c]

,Innings Number,Innings Name,Batsman,Dismissal,Runs,Balls,Minutes,4s,6s,Strike Rate
0,1,Sunrisers Hyderabad,Abhishek Sharma,b Starc,2,5,3,0,0,40.00
1,1,Sunrisers Hyderabad,Travis Head,c †Rahmanullah Gurbaz b Arora,0,1,11,0,0,0.00
2,1,Sunrisers Hyderabad,Rahul Tripathi,c Ramandeep Singh b Starc,9,13,21,1,0,69.23
3,1,Sunrisers Hyderabad,Aiden Markram,c Starc b Russell,20,23,41,3,0,86.95
4,1,Sunrisers Hyderabad,Nitish Kumar Reddy,c †Rahmanullah Gurbaz b Harshit Rana,13,10,14,1,1,130.00
5,1,Sunrisers Hyderabad,Heinrich Klaasen †,b Harshit Rana,16,17,32,1,0,94.11
6,1,Sunrisers Hyderabad,Shahbaz Ahmed,c Narine b Varun,8,7,5,0,1,114.28
7,1,Sunrisers Hyderabad,Abdul Samad,c †Rahmanullah Gurbaz b Russell,4,4,3,0,0,100.00
8,1,Sunrisers Hyderabad,Pat Cummins (c),c Starc b Russell,24,19,28,2,1,126.31
9,1,Sunrisers Hyderabad,Jaydev Unadkat,lbw b Narine,4,11,16,0,0,36.36


In [68]:
display(match_score)

,Innings Number,Innings Name,Score,Overs,Run Rate,Minutes
0,1,Sunrisers Hyderabad,113,18.3,6.10,-
1,2,Kolkata Knight Riders,114/2,10.3,10.85,-


In [69]:
display(did_not_bat)

,Innings Number,Innings Name,Batsman
0,1,Sunrisers Hyderabad,T Natarajan
1,2,Kolkata Knight Riders,Rinku Singh
2,2,Kolkata Knight Riders,Andre Russell
3,2,Kolkata Knight Riders,Ramandeep Singh
4,2,Kolkata Knight Riders,Mitchell Starc
5,2,Kolkata Knight Riders,Vaibhav Arora
6,2,Kolkata Knight Riders,Harshit Rana
7,2,Kolkata Knight Riders,Varun Chakravarthy


In [70]:
display(bowlers_info)

,Innings Number,Innings Name,Bowler,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,Wides,No Balls
0,1,Sunrisers Hyderabad,Mitchell Starc,3,0,14,2,4.66,11,2,0,0,0
1,1,Sunrisers Hyderabad,Vaibhav Arora,3,0,24,1,8.00,9,2,1,4,0
2,1,Sunrisers Hyderabad,Harshit Rana,4,1,24,2,6.00,13,2,1,1,0
3,1,Sunrisers Hyderabad,Sunil Narine,4,0,16,1,4.00,9,0,0,0,0
4,1,Sunrisers Hyderabad,Andre Russell,2.3,0,19,3,7.60,7,1,1,1,0
5,1,Sunrisers Hyderabad,Varun Chakravarthy,2,0,9,1,4.50,7,1,0,0,0
6,2,Kolkata Knight Riders,Bhuvneshwar Kumar,2,0,25,0,12.50,5,2,2,1,0
7,2,Kolkata Knight Riders,Pat Cummins,2,0,18,1,9.00,7,2,1,1,0
8,2,Kolkata Knight Riders,T Natarajan,2,0,29,0,14.50,2,4,1,2,0
9,2,Kolkata Knight Riders,Shahbaz Ahmed,2.3,0,22,1,8.80,7,1,2,0,0


In [65]:
print(winner)

Kolkata Knight Riders


In [ ]:
class Series:
    def __init__(self,ipl_url):
        self.ipl_url = ipl_url
        self.match_links = self.match_link_generator()
        self.match_objects = []
        for match in self.match_links:
            try:
                self.match_objects.append(Score(match))
            except:
                pass
    
    def match_link_generator(self):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }

        response = requests.get(self.ipl_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        match_links = []
        match_boxes = soup.find_all('div')
        for match_box in match_boxes:
            try:
                link_part = match_box.find('a',class_="ds-no-tap-higlight")['href']
                if "indian-premier-league-2024" in link_part:
                    match_link = "https://www.espncricinfo.com" + link_part
                    if match_link not in match_links:
                        match_links.append(match_link)
            except:
                pass

        #print(match_links)
        return match_links
        

In [47]:
ipl = Series("https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/match-schedule-fixtures-and-results")

['https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-royal-challengers-bengaluru-1st-match-1422119/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/punjab-kings-vs-delhi-capitals-2nd-match-1422120/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-3rd-match-1422121/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-lucknow-super-giants-4th-match-1422122/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/gujarat-titans-vs-mumbai-indians-5th-match-1422123/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-punjab-kings-6th-match-1422124/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-gujarat-titans-7t

In [48]:
print(ipl.match_links)

['https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-royal-challengers-bengaluru-1st-match-1422119/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/punjab-kings-vs-delhi-capitals-2nd-match-1422120/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-3rd-match-1422121/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-lucknow-super-giants-4th-match-1422122/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/gujarat-titans-vs-mumbai-indians-5th-match-1422123/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-punjab-kings-6th-match-1422124/full-scorecard', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-gujarat-titans-7t

In [49]:
match_objects = ipl.match_objects

In [50]:
print(match_objects[1].url)

https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/punjab-kings-vs-delhi-capitals-2nd-match-1422120/full-scorecard


In [53]:
for match in match_objects:
    match.printing_scorecard()

Player List:
                           Team              Player
0   Royal Challengers Bengaluru         Virat Kohli
1   Royal Challengers Bengaluru      Faf du Plessis
2   Royal Challengers Bengaluru       Rajat Patidar
3   Royal Challengers Bengaluru       Glenn Maxwell
4   Royal Challengers Bengaluru       Cameron Green
5   Royal Challengers Bengaluru          Anuj Rawat
6   Royal Challengers Bengaluru      Dinesh Karthik
7   Royal Challengers Bengaluru         Karn Sharma
8   Royal Challengers Bengaluru      Alzarri Joseph
9   Royal Challengers Bengaluru        Mayank Dagar
10  Royal Challengers Bengaluru      Mohammed Siraj
11          Chennai Super Kings     Ruturaj Gaikwad
12          Chennai Super Kings     Rachin Ravindra
13          Chennai Super Kings      Ajinkya Rahane
14          Chennai Super Kings      Daryl Mitchell
15          Chennai Super Kings         Shivam Dube
16          Chennai Super Kings     Ravindra Jadeja
17          Chennai Super Kings        Sameer Rizvi

In [45]:
ipl.match_links[0]

'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-royal-challengers-bengaluru-1st-match-1422119/full-scorecard'

In [117]:
players = [['Sunrisers Hyderabad', 'Abhishek Sharma'], ['Sunrisers Hyderabad', 'Travis Head'], 
           ['Sunrisers Hyderabad', 'Rahul Tripathi'], ['Sunrisers Hyderabad', 'Aiden Markram'], 
           ['Sunrisers Hyderabad', 'Nitish Kumar Reddy'], ['Sunrisers Hyderabad', 'Heinrich Klaasen'], 
           ['Sunrisers Hyderabad', 'Shahbaz Ahmed'], ['Sunrisers Hyderabad', 'Abdul Samad'], 
           ['Sunrisers Hyderabad', 'Pat Cummins'], ['Sunrisers Hyderabad', 'Jaydev Unadkat'], 
           ['Sunrisers Hyderabad', 'Bhuvneshwar Kumar'], ['Sunrisers Hyderabad', 'T Natarajan'], 
           ['Kolkata Knight Riders', 'Rahmanullah Gurbaz'], ['Kolkata Knight Riders', 'Sunil Narine'], 
           ['Kolkata Knight Riders', 'Venkatesh Iyer'], ['Kolkata Knight Riders', 'Shreyas Iyer'], 
           ['Kolkata Knight Riders', 'Rinku Singh'], ['Kolkata Knight Riders', 'Andre Russell'], 
           ['Kolkata Knight Riders', 'Ramandeep Singh'], ['Kolkata Knight Riders', 'Mitchell Starc'], 
           ['Kolkata Knight Riders', 'Vaibhav Arora'], ['Kolkata Knight Riders', 'Harshit Rana'], 
           ['Kolkata Knight Riders', 'Varun Chakravarthy']]

dismissals = ['b Starc ', 'c †Rahmanullah Gurbaz b Arora', 'c Ramandeep Singh b Starc', 
              'c Starc b Russell', 'c †Rahmanullah Gurbaz b Harshit Rana', 'b Harshit Rana', 
              'c Narine b Varun', 'c Harshit b Russell', 'c Starc b Russell', 
              'lbw b Narine', 'not out', 'lbw b Shahbaz Ahmed', 'c Shahbaz Ahmed b Cummins', 
              'not out', 'not out', 'run out (Kohli)','c SS Iyer b Starc', 'lbw b Jatinder','st †Rahmanullah Gurbaz b Harshit Rana']

# Step 1: Create a dictionary for team-wise player lookup
team_players = {}
for team, player in players:
    if team not in team_players:
        team_players[team] = []
    team_players[team].append(player)

# Step 2: Extract players involved in dismissals
catchers = []
stumpers = []
runouters = []

for dismissal in dismissals:
    if dismissal.startswith("c "):  # Catchers
        catcher_name = dismissal.split("c ")[1].split(' b ')[0]
        catchers.append(catcher_name)
    elif dismissal.startswith("st "):  # Stumpers
        stumper_name = dismissal.split("st ")[1].split(' b ')[0]
        stumpers.append(stumper_name)
    elif dismissal.startswith("run out ("):  # Stumpers
        runouter_name = dismissal.split("run out (")[1].split(')')[0].split('/')[0]
print("Catchers:", catchers)
print("Stumpers:", stumpers)
print("Runouters:", runouters)

# Step 3: Map names to correct players from the team list
def find_full_name(short_name):
    for team, player in players:
        if "†" in short_name:
            short_name = short_name.split('†')[0]
        if "(c)" in short_name:
            short_name = short_name.split('(c)')[0]
        if short_name in player:
            return (player,dismissal)

        else:
            parts = short_name.split(' ')
            if len(parts) == 1:
                if short_name in player:  # If short name appears in full name
                    return (player,dismissal)
            elif parts[1] in player and player.startswith(parts[0][0]):
                return (player,dismissal)
    return short_name  # If no match is found, return the original name

mapped_catchers = [find_full_name(name) for name in catchers]
mapped_stumpers = [find_full_name(name) for name in stumpers]
mapped_runouters = [find_full_name(name) for name in runouters]

# Step 4: Display results
print("Catchers:", mapped_catchers)
print("Stumpers:", mapped_stumpers)
print("Runouters:", mapped_runouters)

print(team_players)


Catchers: ['†Rahmanullah Gurbaz', 'Ramandeep Singh', 'Star', '†Rahmanullah Gurbaz', 'Narine', 'Harshit', 'Star', 'Shahbaz Ahmed', 'SS Iyer']
Stumpers: ['†Rahmanullah Gurbaz']
Runouters: []
Catchers: [('Abhishek Sharma', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Rinku Singh', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Mitchell Starc', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Abhishek Sharma', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Sunil Narine', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Harshit Rana', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Mitchell Starc', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Shahbaz Ahmed', 'st †Rahmanullah Gurbaz b Harshit Rana'), ('Shreyas Iyer', 'st †Rahmanullah Gurbaz b Harshit Rana')]
Stumpers: [('Abhishek Sharma', 'st †Rahmanullah Gurbaz b Harshit Rana')]
Runouters: []
{'Sunrisers Hyderabad': ['Abhishek Sharma', 'Travis Head', 'Rahul Tripathi', 'Aiden Markram', 'Nitish Kumar Reddy', 'Heinrich Klaasen', 'Shahbaz Ahmed', 'Abdul Samad', 'Pat

In [ ]:
class Series:
    def __init__(self,url):
        self.url = url
        self.match_links = self.match_link_generator()
        self.player_list,self.team_links = self.TeamLinks()
        self.match_objects = []
        for match in self.match_links:
            try:
                self.match_objects.append(Score(match))
            except:
                pass
    
    def match_link_generator(self):


        match_links = []
        match_boxes = soup.find_all('div')
        for match_box in match_boxes:                               
            try:
                link_part = match_box.find('a',class_="ds-no-tap-higlight")['href']
                if "indian-premier-league" in link_part or "ipl-2025" in link_part:
                    match_link = "https://www.espncricinfo.com" + link_part
                    if match_link not in match_links:
                        match_links.append(match_link)
            except:
                pass
        return match_links
    
    def TeamLinks(self):
        #ds-text-compact-s ds-font-bold ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }
        url = "https://www.espncricinfo.com/series/ipl-2025-1449924/squads"
        response = requests.get(url, headers=headers)
        soupteams = BeautifulSoup(response.content, "html.parser")
        teamtable = soupteams.find('div',class_="ds-mb-4")
        teams = teamtable.find_all('a')
        player_list = {}
        team_links = {}
        for team in teams:
            team_name = team['title'].strip().split(' Squad')[0]
            player_list[team_name] = []
            team_links[team_name] = "https://www.espncricinfo.com" + team['href']
            for key in team_links.keys():
                team_url = team_links[key]
                response2 = requests.get(team_url,headers=headers)
                soupplayers = BeautifulSoup(response2.content,"html.parser")
                players= soupplayers.find_all('div',class_="ds-flex ds-flex-row ds-items-center ds-justify-between")
                for player in players:
                    player_name = player.text.strip()
                    if "†" in player_name:
                        player_name = player_name.split('†')[0]
                    if "(c)" in player_name:
                        player_name = player_name.split('(c)')[0]
                    player_list[key].append(player_name)
        return player_list,team_links

In [ ]:
ipl2025 = Series("https://www.espncricinfo.com/series/ipl-2025-1449924/match-schedule-fixtures-and-results")
print(len(ipl2025.player_list))     

TypeError: Series.TeamLinks() missing 1 required positional argument: 'self'

In [119]:
str = 'c & b Harshit Rana'

parts = str.split('c & b ')
print(parts)

['', 'Harshit Rana']


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}
ipl25 = "https://www.cricbuzz.com/cricket-series/9237/indian-premier-league-2025/matches"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

matches = soup.find_all('div',class_='cb-col-75 cb-col')
match_links = []
for match in matches:
    match_link = "https://www.cricbuzz.com" + match.find('a')['href']
    match_link = match_link.replace('live-cricket-scores','live-cricket-scorecard')
    match_links.append(match_link)
print((match_links))

['https://www.cricbuzz.com/live-cricket-scorecard/114960/kkr-vs-rcb-1st-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/114967/srh-vs-rr-2nd-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/114976/csk-vs-mi-3rd-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/114985/dc-vs-lsg-4th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/114987/gt-vs-pbks-5th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/114996/rr-vs-kkr-6th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/115005/srh-vs-lsg-7th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/115012/csk-vs-rcb-8th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/115014/gt-vs-mi-9th-match-indian-premier-league-2025', 'https://www.cricbuzz.com/live-cricket-scorecard/11

In [10]:
players = ['Ruturaj Gaikwad', 'C Andre Siddarth', 'Vansh Bedi', 'Devon Conway', 'MS Dhoni', 'Shaik Rasheed', 'Rahul Tripathi', 'Ravichandran Ashwin', 'Sam Curran', 'Shivam Dube', 'Ramakrishna Ghosh', 'Shreyas Gopal', 'Deepak Hooda', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Rachin Ravindra', 'Vijay Shankar', 'Khaleel Ahmed', 'Nathan Ellis', 'Gurjapneet Singh', 'Mukesh Choudhary', 'Kamlesh Nagarkoti', 'Noor Ahmad', 'Matheesha Pathirana',
'Abishek Porel', 'Harry Brook', 'Faf du Plessis', 'Jake Fraser-McGurk', 'Karun Nair', 'KL Rahul', 'Sameer Rizvi', 'Tristan Stubbs', 'Donovan Ferreira', 'Ajay Mandal', 'Manvanth Kumar', 'Axar Patel', 'Ashutosh Sharma', 'Madhav Tiwari', 'Dushmantha Chameera', 'Kuldeep Yadav', 'Mukesh Kumar', 'T Natarajan', 'Vipraj Nigam', 'Mohit Sharma', 'Mitchell Starc', 'Tripurana Vijay',
'Shubman Gill', 'Anuj Rawat', 'Jos Buttler', 'Kumar Kushagra', 'Sherfane Rutherford', 'Sai Sudharsan', 'M Shahrukh Khan', 'Karim Janat', 'Mahipal Lomror', 'Glenn Phillips', 'Rashid Khan', 'Nishant Sindhu', 'Manav Suthar', 'Rahul Tewatia', 'Washington Sundar', 'Arshad Khan', 'Gerald Coetzee', 'Gurnoor Brar', 'Kulwant Khejroliya', 'Mohammed Siraj', 'Prasidh Krishna', 'Kagiso Rabada', 'Sai Kishore', 'Ishant Sharma', 'Jayant Yadav',
'Quinton de Kock', 'Manish Pandey', 'Rovman Powell', 'Angkrish Raghuvanshi', 'Ajinkya Rahane', 'Rahmanullah Gurbaz', 'Ramandeep Singh', 'Rinku Singh', 'Luvnith Sisodia', 'Moeen Ali', 'Venkatesh Iyer', 'Sunil Narine', 'Anukul Roy', 'Andre Russell', 'Vaibhav Arora', 'Harshit Rana', 'Spencer Johnson', 'Mayank Markande', 'Anrich Nortje', 'Umran Malik', 'Varun Chakravarthy',
'Abdul Samad', 'Ayush Badoni', 'Matthew Breetzke', 'Aryan Juyal', 'David Miller', 'Rishabh Pant', 'Nicholas Pooran', 'Yuvraj Chaudhary', 'Rajvardhan Hangargekar', 'Arshin Kulkarni', 'Aiden Markram', 'Mitchell Marsh', 'Shahbaz Ahmed', 'Akash Deep', 'Akash Singh', 'Avesh Khan', 'Shamar Joseph', 'Mohsin Khan', 'Prince Yadav', 'Digvesh Rathi', 'Ravi Bishnoi', 'Manimaran Siddharth', 'Mayank Yadav',
'Bevon Jacobs', 'Robin Minz', 'Naman Dhir', 'Ryan Rickelton', 'Rohit Sharma', 'Krishnan Shrijith', 'Suryakumar Yadav', 'Raj Bawa', 'Will Jacks', 'Hardik Pandya', 'Mitchell Santner', 'Tilak Varma', 'Ashwani Kumar', 'Trent Boult', 'Jasprit Bumrah', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Vignesh Puthur', 'Satyanarayana Raju', 'Karn Sharma', 'Arjun Tendulkar', 'Reece Topley', 'Lizaad Williams', 'AM Ghazanfar',
'Priyansh Arya', 'Pyla Avinash', 'Harnoor Singh', 'Josh Inglis', 'Shreyas Iyer', 'Prabhsimran Singh', 'Vishnu Vinod', 'Nehal Wadhera', 'Azmatullah Omarzai', 'Aaron Hardie', 'Marco Jansen', 'Glenn Maxwell', 'Musheer Khan', 'Shashank Singh', 'Marcus Stoinis', 'Suryansh Shedge', 'Arshdeep Singh', 'Xavier Bartlett', 'Yuzvendra Chahal', 'Praveen Dubey', 'Lockie Ferguson', 'Harpreet Brar', 'Kuldeep Sen', 'Vijaykumar Vyshak', 'Yash Thakur',
'Sanju Samson', 'Shubham Dubey', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Dhruv Jurel', 'Riyan Parag', 'Nitish Rana', 'Kunal Singh Rathore', 'Vaibhav Suryavanshi', 'Wanindu Hasaranga', 'Jofra Archer', 'Ashok Sharma', 'Tushar Deshpande', 'Fazalhaq Farooqi', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Sandeep Sharma', 'Maheesh Theekshana', 'Yudhvir Singh', 
'Swastik Chikara', 'Tim David', 'Virat Kohli', 'Devdutt Padikkal', 'Rajat Patidar', 'Phil Salt', 'Jitesh Sharma', 'Jacob Bethell', 'Manoj Bhandage', 'Liam Livingstone', 'Mohit Rathee', 'Krunal Pandya', 'Romario Shepherd', 'Abhinandan Singh', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Lungi Ngidi', 'Rasikh Salam', 'Suyash Sharma', 'Swapnil Singh', 'Nuwan Thushara', 'Yash Dayal', 
'Travis Head', 'Ishan Kishan', 'Heinrich Klaasen', 'Sachin Baby', 'Aniket Verma', 'Abhishek Sharma', 'Abhinav Manohar', 'Kamindu Mendis', 'Wiaan Mulder', 'Nitish Kumar Reddy', 'Atharva Taide', 'Brydon CarseWithdrawn', 'Rahul Chahar', 'Pat Cummins', 'Eshan Malinga', 'Mohammed Shami', 'Harshal Patel', 'Simarjeet Singh', 'Jaydev Unadkat', 'Adam Zampa', 'Zeeshan Ansari']
ipl24 = "https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches"
ipl25 = "https://www.cricbuzz.com/cricket-series/9237/indian-premier-league-2025/matches"
wrong_list = []
for match in range(1,75):   
    try: 
        print("Match Number",match)
        players_list = match_squads_generator(ipl25,match)
        #print(players_list)
        for team in players_list.keys():
            for player in players_list[team]:
                if player not in players and player not in wrong_list:
                    print(player)
                    wrong_list.append(player)
    except:
        print("Match Abandoned?:",match)
print(wrong_list)


Match Number 1
Philip Salt
Match Number 2
Brydon Carse
Yudhvir Singh Charak
Match Number 3
Andre Siddarth C
Match Number 4
Darshan Nalkande
Ajay Jadav Mandal
Manvanth Kumar L
Himmat Singh
Digvesh Singh
Match Number 5
Shahrukh Khan
Ravisrinivasan Sai Kishore
Match Number 6
Match Number 7
Match Number 8
Match Number 9
Match Number 10
Match Number 11
Match Number 12
Match Number 13
Match Number 14
Match Number 15
Match Number 16
Match Number 17
Match Number 18
Match Abandoned?: 18
Match Number 19
Match Number 20
Match Number 21
Match Number 22
Match Abandoned?: 22
Match Number 23
Match Abandoned?: 23
Match Number 24
Match Number 25
Match Number 26
Match Number 27
Match Number 28
Match Abandoned?: 28
Match Number 29
Match Number 30
Match Number 31
Match Abandoned?: 31
Match Number 32
Match Number 33
Match Number 34
Match Number 35
Match Number 36
Match Number 37
Match Number 38
Match Abandoned?: 38
Match Number 39
Match Number 40
Match Number 41
Match Number 42
Match Number 43
Match Number

In [11]:
print(wrong_list)

['Philip Salt', 'Brydon Carse', 'Yudhvir Singh Charak', 'Andre Siddarth C', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Himmat Singh', 'Digvesh Singh', 'Shahrukh Khan', 'Ravisrinivasan Sai Kishore']


In [8]:
def extract_team_players(players_list, impact_sub_class):
    team = []
    try:
        for i in range(16):
            if i < 26:
                player = clean_player_name(players_list[i].text)
                team.append(player)
        return team
    except:
        pass

def clean_player_name(text):
    return text.strip().split('  ')[0].split('(C & WK)')[0].split(' (C)')[0].split(' (WK)')[0].strip()

def match_squads_generator(ipl_url, match_number):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    response = requests.get(ipl_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    matches = soup.find_all('div', class_='cb-col-75 cb-col')
    match = matches[match_number - 1]
    match_link = "https://www.cricbuzz.com" + match.find('a')['href']
    
    if 'live-cricket-scores' in match_link:
        match_link = match_link.replace('live-cricket-scores', 'cricket-match-squads')
    elif 'cricket-scores' in match_link:
        match_link = match_link.replace('cricket-scores', 'cricket-match-squads')

    response_squads = requests.get(match_link, headers=headers)
    soup_squads = BeautifulSoup(response_squads.content, "html.parser")
    #print(soup_squads.prettify())
    team_matchup = soup_squads.find('h1',class_='cb-nav-hdr cb-font-18 line-ht24').text.strip()
    team1_name = team_matchup.split(' vs ')[0]
    team2_name = team_matchup.split(' vs ')[1].split(',')[0].strip()

    required_classes_team1 = {'cb-col', 'cb-col-100', 'pad10', 'cb-player-card-left'}

    players_team1 = [
    a for a in soup_squads.find_all('a', class_=True) 
    if required_classes_team1.issubset(set(a['class']))
]
    required_classes_team2 = {'cb-col', 'cb-col-100', 'pad10', 'cb-player-card-right'}

    players_team2 = [
    a for a in soup_squads.find_all('a', class_=True) 
    if required_classes_team2.issubset(set(a['class']))
]
    #players_team2 = soup_squads.find_all('a', class_='cb-col cb-col-100 pad10 cb-player-card-right')

    team1 = extract_team_players(players_team1, "cb-plus-match-change-icon cb-bg-min cb-match-change-left")
    team2 = extract_team_players(players_team2, "cb-plus-match-change-icon cb-bg-min cb-match-change-right")
    teams = {team1_name:team1,team2_name:team2}

    return teams

ipl24 = "https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches"
ipl25 = "https://www.cricbuzz.com/cricket-series/9237/indian-premier-league-2025/matches"
players = match_squads_generator(ipl24,25)
print(players)

{'Mumbai Indians': ['Rohit Sharma', 'Ishan Kishan', 'Tilak Varma', 'Hardik Pandya', 'Tim David', 'Romario Shepherd', 'Mohammad Nabi', 'Shreyas Gopal', 'Jasprit Bumrah', 'Gerald Coetzee', 'Akash Madhwal', 'Suryakumar Yadav', 'Dewald Brevis', 'Naman Dhir', 'Nehal Wadhera', 'Harvik Desai'], 'Royal Challengers Bengaluru': ['Virat Kohli', 'Faf du Plessis', 'Will Jacks', 'Rajat Patidar', 'Glenn Maxwell', 'Dinesh Karthik', 'Mahipal Lomror', 'Reece Topley', 'Vijaykumar Vyshak', 'Mohammed Siraj', 'Akash Deep', 'Suyash Prabhudessai', 'Saurav Chauhan', 'Swapnil Singh', 'Rajan Kumar', 'Karn Sharma']}


In [48]:
from fuzzywuzzy import process

player_names = ["Andre Siddharth C", "Phillip Salt","Sandeep Sharma", "Karn Sharma","Ravindra Jadeja", "Rachin Ravindra","Rinku Singh","Mahendra Singh Dhoni"]
input_name = "R Ravindra"

best_match, score = process.extractOne(input_name, player_names)

if score > 20:
    print(f"Matched: {best_match} (Score: {score})")
else:
    print("No close match found.")


Matched: Ravindra Jadeja (Score: 86)


In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}
url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/match-schedule-fixtures-and-results"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
match_links = []
match_boxes = soup.find_all('div')
for match_box in match_boxes:     
    match_block = match_box.find('a',class_="ds-no-tap-higlight")
    try:
        link_part = match_block['href']
        abandoned = match_block.find('p',class_='ds-text-tight-s ds-font-medium ds-line-clamp-2 ds-text-typo').text.strip()
        if 'bandoned without a ball bowled' not in abandoned or 'bandoned with a toss' not in abandoned:
            if "indian-premier-league" in link_part or "ipl-2025" in link_part and 'full-scorecard' in link_part:
                match_link = "https://www.espncricinfo.com" + link_part
                if match_link not in match_links:
                    match_links.append(match_link)
    except:
        pass

In [46]:
c = "Phil"
d = "Philip"
if c in d:
    print("Yeah boi")

Yeah boi


In [13]:
pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
    --------------------------------------- 0.0/1.6 MB 217.9 kB/s eta 0:00:08
   -- ------------------------------------- 0.1/1.6 MB 508.4 kB/s eta 0:00:04
   ----- ---------------------------------- 0.2/1.6 MB 1.1 MB/s eta 0:00:02
   ---------------- ----------------------- 0.7/1.6 MB 2.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.6 MB 3.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.4/1.6 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.3 MB/s eta 0:00:00


In [34]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def dismissals_scraper(soup,innings_id):
    innings = soup.find('div',id=innings_id)
    innings_name = innings.find('span').text.split(' Innings')[0]
    #print(innings_name)
    player_batting = innings.find_all('div',class_="cb-col cb-col-100 cb-scrd-itms")
    dismissals = []
    for player in player_batting:
        dismissal = player.find('span',class_='text-gray')
        if dismissal:
            dismissal = dismissal.text
            dismissals.append(dismissal)
    return innings_name,dismissals
def match_dismissals_output(ipl_url, match_number):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    response = requests.get(ipl_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    matches = soup.find_all('div', class_='cb-col-75 cb-col')
    match = matches[match_number - 1]
    match_link = "https://www.cricbuzz.com" + match.find('a')['href']
    
    if 'live-cricket-scores' in match_link:
        match_link = match_link.replace('live-cricket-scores', 'live-cricket-scorecard')
    elif 'cricket-scores' in match_link:
        match_link = match_link.replace('cricket-scores', 'live-cricket-scorecard')

    response_score = requests.get(match_link, headers=headers)
    soup_score = BeautifulSoup(response_score.content, "html.parser")
    innings1,dismissals1 = dismissals_scraper(soup_score,"innings_1")
    innings2,dismissals2 = dismissals_scraper(soup_score,"innings_2")   
    dismissals = {innings2:dismissals1,innings1:dismissals2}
    return dismissals

ipl24 = "https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches"
ipl25 = "https://www.cricbuzz.com/cricket-series/9237/indian-premier-league-2025/matches"
dismissals = match_dismissals_output(ipl24,63)
print(dismissals)

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
def dismissals_final_generator(ipl_url,game_number):
    players = match_squads_generator(ipl_url,game_number)
    print("Players:",players)
    dismissals = match_dismissals_output(ipl_url,game_number)
    print("Dismissals:",dismissals)
    # Step 2: Extract players involved in dismissals
    catchers = {}
    stumpers = {}
    runouters = {}
    lbw = {}
    bowled = {}

    for key in dismissals.keys():
        for dismissal in dismissals[key]:
            if dismissal.startswith("c and b "):  # Catchers
                catcher_name = dismissal.split("c and b ")[1]
                catchers.setdefault(key, []).append(catcher_name)
            elif dismissal.startswith("c "):  # Catchers
                catcher_name = dismissal.split("c ",1)[1].split(' b ')[0]
                catchers.setdefault(key, []).append(catcher_name)
            elif dismissal.startswith("st "):  # Stumpers
                stumper_name = dismissal.split("st ",1)[1].split(' b ')[0]
                stumpers.setdefault(key, []).append(stumper_name)
            elif dismissal.startswith("run out ("):  # Stumpers
                runouter_name = dismissal.split("run out (",1)[1].split(')')
                #print(runouter_name)
                runouter_name = runouter_name[-2].split('/')[0]
                runouters.setdefault(key, []).append(runouter_name)
            elif dismissal.startswith("b "):  # Stumpers
                bowled_name = dismissal.split("b ",1)[1]
                bowled.setdefault(key, []).append(bowled_name)

    # Step 3: Map names to correct players from the team list
    def find_full_name(team,short_name):
        for player in players[team]:
            if "†" in short_name:
                short_name = short_name.split('†')[1]
            if "(c)" in short_name:
                short_name = short_name.split('(c)')[0]
            if "(sub)" in short_name:
                return None
            if short_name == player: 
                return player
            else:
                parts = short_name.split(' ')
                if len(parts) == 1:
                    if short_name in player:  # If short name appears in full name
                        return player
                elif len(parts) == 2:
                    if parts[1] in player:
                        if parts[0] in player:
                            return player
                elif parts[-1] in player and player.startswith(parts[0][0]):
                    return player
        #print(short_name,"ne gaand maar di")
        return short_name  # If no match is found, return the original name

    mapped_catchers = [full_name for team, names in catchers.items() 
    for name in names 
    if (full_name := find_full_name(team, name)) is not None]
    mapped_stumpers = [full_name for team, names in stumpers.items() 
    for name in names 
    if (full_name := find_full_name(team, name)) is not None]
    mapped_runouters = [   full_name for team, names in runouters.items() 
    for name in names 
    if (full_name := find_full_name(team, name)) is not None]

    # Step 4: Display results
    print("Catchers:", mapped_catchers)
    print("Stumpers:", mapped_stumpers)
    print("Runouters:", mapped_runouters)
            

In [254]:
ipl24 = "https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches"
dismissals_final_generator(ipl24,72)

Players: {'Rajasthan Royals': ['Yashasvi Jaiswal', 'Tom Kohler-Cadmore', 'Sanju Samson', 'Riyan Parag', 'Dhruv Jurel', 'Rovman Powell', 'Ravichandran Ashwin', 'Trent Boult', 'Avesh Khan', 'Sandeep Sharma', 'Yuzvendra Chahal', 'Shimron Hetmyer'], 'Royal Challengers Bengaluru': ['Virat Kohli', 'Faf du Plessis', 'Rajat Patidar', 'Cameron Green', 'Glenn Maxwell', 'Dinesh Karthik', 'Mahipal Lomror', 'Karn Sharma', 'Yash Dayal', 'Mohammed Siraj', 'Lockie Ferguson', 'Swapnil Singh']}
Dismissals: {'Rajasthan Royals': ['c (sub)Donovan Ferreira b Chahal', 'c Rovman Powell b Boult', 'c Rovman Powell b Ashwin', 'c Riyan Parag b Avesh Khan', 'c Dhruv Jurel b Ashwin', 'c Rovman Powell b Avesh Khan', 'c Yashasvi Jaiswal b Avesh Khan', 'not out', 'c Rovman Powell b Sandeep Sharma'], 'Royal Challengers Bengaluru': ['c Karthik b Green', 'b Lockie Ferguson', 'st Karthik b Karn Sharma', 'b Siraj', 'run out (Kohli/Green)', 'c du Plessis b Siraj', 'not out', 'not out']}
Catchers: ['Rovman Powell', 'Rovman P

In [276]:
#str = "https://www.espncricinfo.com/series/ipl-2025-1449924/kolkata-knight-riders-vs-royal-challengers-bengaluru-1st-match-1473438/live-cricket-score"
def match_number_generator(match_link):
    parts = (match_link.split('/')[-2].split('-')[-3:-1])
    if parts[1] == 'match':
        match_number = parts[0][:-2]
    elif parts[0] == 'qualifier':
        if parts[1] == 1:
            match_number = 71
        elif parts[1] == 2:
            match_number = 73
    elif parts[1] == 'eliminator':
        match_number = 72
    elif parts[1] == 'final':
        match_number = 74
    print(match_number)
match_link = 'https://www.espncricinfo.com/series/ipl-2025-1449924/tba-vs-tba-final-1473509/live-cricket-score'
match_number_generator(match_link)

74


In [4]:
import pandas as pd

# Sample DataFrame
data = {
    'Batsman': ['Sachin Tendulkar', 'Virat Kohli', 'MS Dhoni', 'Rohit Sharma'],
    'Runs': [18426, 13848, 10500, 9825]
}

df = pd.DataFrame(data)

# Given batsman
batsman_name = 'Virat Kohli'

# Find runs
runs = df.loc[df['Batsman'] == batsman_name, 'Runs'].values

print(runs)


[13848]


In [7]:
round(123.435,2)

123.44